# Max jump speed

Validates the maximum jump speed.  Uses the SSScoring API and raw dataframe operations to compare the results.

In [ ]:
import warnings

import pandas as pd
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue if running in Lucyfer/Dockerized Jupyter 

In [ ]:
# SSScoring
from ssscoring.calc import convertFlySight2SSScoring
from ssscoring.calc import getFlySightDataFromCSVFileName
from ssscoring.calc import processJump
from ssscoring.constants import MPS_2_KMH

## Lets get the working datasets

In [ ]:
SAMPLE_TRACK_FILE = 'data/24WSC_SP_R3_643.CSV'
dropZoneAltMSLMeters = 3.0 # Crystal Coast Skydiving

orgData = pd.read_csv(SAMPLE_TRACK_FILE, sep=',', skiprows=(1,1), index_col=False)
data, tag = getFlySightDataFromCSVFileName(SAMPLE_TRACK_FILE)
data = convertFlySight2SSScoring(data, altitudeDZMeters=dropZoneAltMSLMeters)

## Identical dataset length

In [ ]:
print('orgData len = %d, data len = %d' % (len(orgData), len(data)))

### Confirm that both data files have the same values using the more meaningful SSScore column names

In [ ]:
orgData.velD.equals(data.vMetersPerSecond)

## Get the absolute max speed using SSScoring and the original FlySight data

In [ ]:
jumpResult = processJump(data)

v1 = orgData[orgData.velD > 100.0].velD  # Get all the data points where velD > 100.0 m/s
v2 = data[data.vMetersPerSecond > 100.0].vMetersPerSecond  # Get all the data points where vMetersPerSecond > 100.0 m/s
pd.DataFrame(
{
    'SSScoring max': [ jumpResult.maxSpeed, ],
    'FlySight vH max': [ MPS_2_KMH*v1.max(), ],
    'SSScoring vH max': [ MPS_2_KMH*v2.max(), ],
})

In [ ]:
print(jumpResult.score, jumpResult.maxSpeed, MPS_2_KMH*v2.max())

<h2><span style='color: #f00'>None of these coincide with the continental speed of 535.41 km/h filed for record at the 2024 World Cup.</span>  The max speed of 533.16 is reported over the full data set.  SSScore reports the absolute max speed within the scoring window.  That means that the absolute max if we follow the naïve FlySightViewer eyeballing method is estimated outside of the scoring window.</h2>

<img src='https://eugeneciurana.com/personal/images/Nik-jump-04-FS-max-speed.png' width='50%' height='50%'>